In [24]:
from IPython.core.display import HTML
def css_styling():
    styles = open("styles/tma4215.css", "r").read()
    return HTML(styles)

css_styling() #Endrer stylingen til notebooken

# Modellering av vær

### Oppgave

Benytt åpne værdata på nettet til å lage en stokastisk værmodell som kan predikere om det blir sol, regn eller oppholdsvær et visst antall dager frem i tid. Beregn deretter hvor nøyaktig modellen din er på et nytt sett data (altså data som ikke har vært brukt til å lage modellen). Oppgaven er relativt åpen ettersom dette er et eget prosjekt på egenhånd der hensikten er å lære seg dataanalyse.

### Utfordringer underveis



##### Finne og hente ut data
Var nødt til å få tak i store mengder værdata for å kunne lage modellen. Fant tidlig ut at metereologisk insitutt har frie data liggende ute. Var derimot ikke like tydelig hvordan disse skulle hentes ut. Etter litt leting fant jeg imidlertid at man kunne hente ut data direkte fra værstasjoner gjennom nettsiden [frost.met.no](https://frost.met.no/index.html). Etter å ha opprettet bruker der kunne jeg da hente værdata rett inn i jupyter notebook-filen.

Måtte deretter lære meg syntaxen og oppsettet til systemet Frost API for å kunne hente ut riktig data. Ønsket å hente ut antall solskinnstimer siste døgnet og mengden nedbør siste døgnet.

##### Bearbeide dataen

1. Fikk flere målinger for regnmengde per dag. **Løsning:** lese seg opp på Frost og endre parameterene når dataen hentes ut.

2. Manglet målinger flere steder. **Løsning:** Fjerne hver dag der det manglet minst en måling.

3. Dette ga et nytt problem siden jeg da ikke hadde kun sammenhengende data. **Løsning:** legge inn en sjekk hver gang jeg trengte sammenhengende data, som deretter automatisk hoppet frem til der dataen var sammenhengende nok.

##### Komplisert modell
Ettersom den endelige modellen er litt komplisert å implementere, gjøres det trinnvis. Lager derfor først en forenklet modell og tester den.

### Koden

Importerer nå de nødvendige bibliotekene og lagrer personlig Frost-id for å koble seg til serverne dems.

In [59]:
import requests
import pandas as pd
import numpy as np
# Insert your own client ID here
client_id = 'd52bbcb0-6576-44fc-9a71-a55a49b61376'

Nedenfor er koden for å koble seg til Frost API-serverne og dermed hente ut historiske data fra meteorologisk institutt. Den gjør i tillegg litt databehandling. Fjerner en del unødvendige kolonner med informasjon og konverterer datoene til en form Python forstår. Returnerer en DataFrame fra biblioteket Pandas.

In [69]:
"""
Kjente værstasjoner:
"SN18700" - Blindern
"""
def getData(timeperiod, client_id, sources = "SN18700"): 
    # Define endpoint and parameters
    endpoint = 'https://frost.met.no/observations/v0.jsonld'
    parameters = {
        'sources': sources, # Får ut data fra en sensor på Oslo - Blindern
        'elements': 'sum(duration_of_sunshine P1D), sum(duration_of_sunshine P1D)',
        'referencetime': timeperiod, 
        'levels' : 'default', "timeoffsets" : "default", #only get one timeseries each day

    }
    # Issue an HTTP GET request
    r = requests.get(endpoint, parameters, auth=(client_id,''), )
    # Extract JSON data
    json = r.json()
    
    # Check if the request worked, print out any errors
    if r.status_code == 200:
        data = json['data']
        #print('Data retrieved from frost.met.no!')
    else:
        print('Error! Returned status code %s' % r.status_code)
        print('Message: %s' % json['error']['message'])
        print('Reason: %s' % json['error']['reason'])
    
    
    df = pd.DataFrame()
    for i in range(len(data)):
        row = pd.DataFrame(data[i]['observations'])
        row['referenceTime'] = data[i]['referenceTime']
        row['sourceId'] = data[i]['sourceId']
        df = df.append(row)

    df = df.reset_index()
    
    # These additional columns will be kept
    columns = ['sourceId','referenceTime','elementId','value','unit','timeOffset']
    df_edited = df[columns].copy()
    # Convert the time value to something Python understands
    df_edited['referenceTime'] = pd.to_datetime(df_edited['referenceTime'])

    
    return df_edited

Hjelpefunksjoner:

In [117]:
#Hjelpefunksjon for å lage tekststrenger med datoer på en form Frost API aksepterer.
def createDate(start, end):
    string = str(start[2]) + "-" + str(start[1]) + "-" + str(start[0]) + "/"
    string += str(end[2]) + "-" + str(end[1]) + "-" + str(end[0])
    
    return string


def createTimeperiods(startYear, endYear, startDate = [1,4], endDate = [30,7]):
    timeperiods = []
    for year in range(startYear, endYear + 1):
        timeperiods.append(createDate([startDate[0], startDate[1], year], [endDate[0], endDate[1], year]))
    return timeperiods

Nedenfor er en funksjon for å hente ut data fra en valgt tidsperiode hvert år. Kan definere fra hvilke år man ønsker å hente ut data. Funksjonen samler alle dataene i et Pandas DataFrame objekt.

In [120]:
#Henter ut data fra en valgt tidsperiode hvert år. Velger selv start- og slutt-år.
def getDataMultipleYears(startYear, endYear):
    frames = []
    for timeperiod in createTimeperiods(startYear, endYear):
        frames.append(getData(timeperiod, client_id))

    return pd.concat(frames, ignore_index = True)

df_edited = getDataMultipleYears(1990, 2015)

Lagrer værdataene til en fil:

df_edited.to_csv('weatherData.csv', index=False)

Slik ser datamengden ut nå:

In [77]:
df_edited

,sourceId,referenceTime,elementId,value,unit,timeOffset
0,SN18700:0,1990-04-01,sum(duration_of_sunshine P1D),2.2,hours,PT0H
1,SN18700:0,1990-04-01,sum(precipitation_amount P1D),0.0,mm,PT6H
2,SN18700:0,1990-04-02,sum(duration_of_sunshine P1D),0.0,hours,PT0H
3,SN18700:0,1990-04-02,sum(precipitation_amount P1D),0.1,mm,PT6H
4,SN18700:0,1990-04-03,sum(duration_of_sunshine P1D),0.8,hours,PT0H
5,SN18700:0,1990-04-03,sum(precipitation_amount P1D),19.4,mm,PT6H
6,SN18700:0,1990-04-04,sum(duration_of_sunshine P1D),8.1,hours,PT0H
7,SN18700:0,1990-04-04,sum(precipitation_amount P1D),4.3,mm,PT6H
8,SN18700:0,1990-04-05,sum(duration_of_sunshine P1D),0.6,hours,PT0H
9,SN18700:0,1990-04-05,sum(precipitation_amount P1D),0.0,mm,PT6H


Sjekker at det ikke er noen målinger med unaturlig høye verdier for antall timer solskinn.

In [79]:
df_edited.loc[(df_edited["elementId"] == "sum(duration_of_sunshine P1D)") & (df_edited["value"] > 17)]

,sourceId,referenceTime,elementId,value,unit,timeOffset
3929,SN18700:0,2006-06-15,sum(duration_of_sunshine P1D),17.1,hours,PT0H
4169,SN18700:0,2007-06-15,sum(duration_of_sunshine P1D),17.1,hours,PT0H
4531,SN18700:0,2009-04-17,sum(duration_of_sunshine P1D),18.4,hours,PT0H
4665,SN18700:0,2009-06-23,sum(duration_of_sunshine P1D),17.1,hours,PT0H
4667,SN18700:0,2009-06-24,sum(duration_of_sunshine P1D),17.1,hours,PT0H
4673,SN18700:0,2009-06-27,sum(duration_of_sunshine P1D),17.1,hours,PT0H
4900,SN18700:0,2010-06-21,sum(duration_of_sunshine P1D),17.1,hours,PT0H


Funksjonen nedenfor bearbeider dataene slik at de kommer på at mer brukbart format. Lager et nytt DataFrame objekt der hver dag med gyldige målinger lagres som en rad med verdier for mengde solskinn og regn. Fjerner alle dager der det ikke finnes fullstendige målinger.

In [83]:
def createDays(df_edited):
    lstColumns = ["Day", "Date", "Sunshine", "unit S", "Rain", "unit R"] #liste med kolonnenavn
    df_days = pd.DataFrame() #dataframe med målinger for de ulike dagene
    rows, col = df_edited.shape
    i = 1
    countDay = 0 #holder styr på hvor mange dager det har gått siden start og dermed om man hopper ov
    while (i < rows):
        if(df_edited.iloc[i,1] != df_edited.iloc[i-1, 1]):
            try:
                countDay += (df_edited.iloc[i+1,1] - df_edited.iloc[i,1]).days #beregner hvilken dag neste måling er fra
            except:
                break #det er siste måling og den er ugyldig, avslutt program

            i += 1
            continue
        date, sunshine, rain = df_edited.iloc[i,1], df_edited.iloc[i,3], df_edited.iloc[i-1,3]

        df_days = df_days.append(pd.Series([countDay, date,sunshine, "hours", rain, "mm"]), ignore_index = True)
        try:
            countDay += (df_edited.iloc[i+2,1] - df_edited.iloc[i,1]).days #beregner hvilken dag neste måling er fra
        except:
            break #det er siste måling og den er ugyldig, avslutt program
        i += 2
        
    df_days.columns = lstColumns
    return df_days

Nedenfor vises de fem første radene i det nye DataFrame objektet.

In [82]:
df_days = createDays(df_edited)
df_days.head()

,Day,Date,Sunshine,unit S,Rain,unit R
0,0.0,1990-04-01,0.0,hours,2.2,mm
1,1.0,1990-04-02,0.1,hours,0.0,mm
2,2.0,1990-04-03,19.4,hours,0.8,mm
3,3.0,1990-04-04,4.3,hours,8.1,mm
4,4.0,1990-04-05,0.0,hours,0.6,mm


## Modell 1

Ønsker å bruke stokastisk modellering til å beregne været. Lager først en litt forenklet versjon som kun bruker informasjon fra dagen før til å beregne de neste dagene.

Definerer tre tilstander:

0 - Sol

1 - Overskyet

2 - Regn

Ønsker å finne en matrise som beskriver overgangen fra dag til dag. Har her begrenset meg til månedene mai, juni og juli ettersom sannsynlighetene endrer seg gjennom året.

In [85]:
def createDfSimpleStates(df_days, limit_rain = 2, limit_sun = 3):
    df_states_simple = df_days.copy()
    numbRows, numbCols = df_states_simple.shape
    df_states_simple.insert(2, "State", np.zeros(numbRows))
    for i in range(numbRows):
        if df_days.iloc[i,4] > limit_rain:
            state = 2
        elif df_days.iloc[i,2] > limit_sun:
            state = 0
        else:
            state = 1
        df_states_simple.iloc[i,2] = state
    
    return df_states_simple

Funksjonen over legger til en ny kolonne som beskriver hvilken tilstand hver dag befinner seg i. Nedenfor vises de første 10 radene av dataen.

In [87]:
df_states_simple = createDfSimpleStates(df_days)
df_states_simple.head(10)

,Day,Date,State,Sunshine,unit S,Rain,unit R
0,0.0,1990-04-01,2.0,0.0,hours,2.2,mm
1,1.0,1990-04-02,1.0,0.1,hours,0.0,mm
2,2.0,1990-04-03,0.0,19.4,hours,0.8,mm
3,3.0,1990-04-04,2.0,4.3,hours,8.1,mm
4,4.0,1990-04-05,1.0,0.0,hours,0.6,mm
5,5.0,1990-04-06,2.0,0.0,hours,9.0,mm
6,6.0,1990-04-07,2.0,0.0,hours,11.3,mm
7,7.0,1990-04-08,2.0,0.0,hours,10.3,mm
8,8.0,1990-04-09,2.0,0.0,hours,4.9,mm
9,9.0,1990-04-10,2.0,0.0,hours,6.6,mm


Funksjonen nedenfor beregner sannsynlighetene for matrisen. Altså for eksempel sannsynligheten for at i morgen regner det, gitt at det var sol i dag.

In [88]:
def getProbsSimple(df):
    A = np.array([[0,0,0], 
                  [0,0,0], 
                  [0,0,0]], dtype = float)
    numbRows, numbCols = df.shape
    prevState = int(df.iloc[0, 2])
    for i in range(2, numbRows):
        state = int(df.iloc[i, 2])
        if (df.iloc[i, 0] - df.iloc[i-1, 0]) == 1:
            A[prevState, state] += 1
        prevState = state
        
    for i in range(len(A)):
        A[i,:] = A[i,:] / np.sum(A[i,:])
    
    return A

Hjelpefunksjoner for å formattere matriser pent når de skal printes.

In [89]:
def styleMatrix(A, states):
    A_pretty = pd.DataFrame(A)
    A_pretty.columns = states
    A_pretty.index = (states)
    return A_pretty

def color_positive_blue(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'blue' if val > 0 else 'black'
    return 'color: %s' % color

Den beregnede matrisen for sannsynlighetene til de ulike overgangene mellom tilstandene blir slik:

In [90]:
A = getProbsSimple(df_states_simple)
styleMatrix(A, ["S (0)", "O (1)", "R (2)"])

,S (0),O (1),R (2)
S (0),0.261062,0.128319,0.610619
O (1),0.201531,0.224490,0.573980
R (2),0.040148,0.125981,0.833872


Hjelpefunksjoner som brukes i funksjonen som tester datasettene.

In [121]:
#Hjelpefunksjon som forskyver en liste n plasser. De første n-verdiene blir satt til å være minus 1
def leftShiftList(lst, n):
    for i in range(len(lst)-1, n-1, -1):
        lst[i] = lst[i-n]

    for i in range(min(n, len(lst))):
        lst[i] = -1
    
    return lst

def multiplyArray(A, n):
    predState = A
    for i in range(n):
        predState = predState @ A
    
    return predState

Funksjonen nedenfor beregner hvor godt modellen klar spå fremtiden. Den returnerer en liste med andelen riktige spådommer for 1,2,...,n dager frem i tid.

In [94]:
def testProbsSimple(A, df, daysIntoFuture):
    
    numbRows, numbCols = df.shape
    
    count = np.zeros(daysIntoFuture)
    correct = np.zeros(daysIntoFuture)
    prevStates = np.ones(daysIntoFuture) * (-1)
    for i in range(numbRows):
        currState = int(df.iloc[i, 2])
        
        #print("currState =", currState)
        
        for j in range(len(prevStates)):
            predMatrix = multiplyArray(A, j)
            predState = np.argmax(predMatrix[int(prevStates[j])])
            
            #print("j =", j, "gir:", predState)
            
            if prevStates[j] == -1:
                break
            
            if currState == predState:
                correct[j] += 1
            
            count[j] += 1
        
        try:
            diff = df.iloc[i+1,1] - df.iloc[i,1]
            leftShiftList(prevStates, diff.days)
        except:
            break #det er siste måling
        
        prevStates[0] = currState
        """
        print(prevStates)
        print(correct)
        print(count, "\n")
        """
    
    return correct / count

Funksjon som kjapt henter ut testdata for nye år. Bruker funksjonene ovenfor.

In [96]:
def quickDfSimpleStates(startYear, endYear):
    df_edited = getDataMultpleYears(startYear, endYear)
    df_days = createDays(df_edited)
    return df_days

testDf = createDfSimpleStates(quickDfSimpleStates(2015, 2019))

I tabellen nedenfor finner man andelen riktige spådommer for hennholdsvis 1,2 og 3 dager frem i tid.

In [113]:
pd.DataFrame(testProbsSimple(A, testDf, 3), columns = ["Andel riktig"], index = [1,2,3])

,Andel riktig
1,0.814070
2,0.812183
3,0.811966


## Modell 2

Videreutvikler nå til en litt mer avansert modell som benytter informasjon fra de siste to dagene til å beregne tilstanden neste dag.

Definerer da 9 tilstander:

0 - SS - Sol i går, sol i dag

1 - SO - Sol i går, overskyet i dag

2 - SR - Sol i går, regn i dag

3 - OS - Overskyet i går, regn i dag

4 - OO - Overskyet i går, overskyet i dag

5 - OR - Overskyet i går, regn i dag

6 - RS - Regn i går, sol i dag

7 - RO - Regn i går, overskyet i dag

8 - RR - Regn i går, regn i dag

Ønsker nå å finne den nye matrisen som beskriver disse overgangene.

In [102]:
#Hjelpefunksjon for å beregne de nye tilstandene
def getSimpleState(sun, rain, limit_sun, limit_rain):
    if rain > limit_rain:
        state = 2
    elif sun >= limit_sun:
        state = 0
    else:
        state = 1
    return state

In [103]:
def createDfStates(df_days, limit_rain = 2, limit_sun = 3):
    df_states = df_days.copy()
    numbRows, numbCols = df_states.shape
    df_states.insert(2, "State", np.zeros(numbRows))
    simpleStates = [-1,-1]
    for i in range(1, numbRows):
        if (df_states.iloc[i, 0] - df_states.iloc[i-1, 0]) == 1:
            simpleStates[1] = getSimpleState(df_days.iloc[i-1,2], df_days.iloc[i-1,4], limit_sun, limit_rain)
            simpleStates[0] = getSimpleState(df_days.iloc[i,2], df_days.iloc[i,4], limit_sun, limit_rain)
            state = simpleStates[1] * 3 + simpleStates[0]
            df_states.iloc[i,2] = state
            
        else:
            df_states.iloc[i,2] = -1
            
    
    
    return df_states

Funksjonen over legger til en ny kolonne som beskriver hvilken tilstand hver dag befinner seg i. Nedenfor vises de første 10 radene av dataen.

In [105]:
df_states = createDfStates(df_days)
df_states.head(10)

,Day,Date,State,Sunshine,unit S,Rain,unit R
0,0.0,1990-04-01,0.0,0.0,hours,2.2,mm
1,1.0,1990-04-02,7.0,0.1,hours,0.0,mm
2,2.0,1990-04-03,3.0,19.4,hours,0.8,mm
3,3.0,1990-04-04,2.0,4.3,hours,8.1,mm
4,4.0,1990-04-05,7.0,0.0,hours,0.6,mm
5,5.0,1990-04-06,5.0,0.0,hours,9.0,mm
6,6.0,1990-04-07,8.0,0.0,hours,11.3,mm
7,7.0,1990-04-08,8.0,0.0,hours,10.3,mm
8,8.0,1990-04-09,8.0,0.0,hours,4.9,mm
9,9.0,1990-04-10,8.0,0.0,hours,6.6,mm


Funksjonen nedenfor beregner sannsynlighetene for matrisen. Altså for eksempel sannsynligheten for at i morgen regner det, gitt at det var sol i dag og overskyet i går.

In [107]:
def getProbs(df):
    A = np.zeros((9,9), dtype = float)
    numbRows, numbCols = df.shape
    prevState = int(df.iloc[1, 2])
    for i in range(2, numbRows):
        state = int(df.iloc[i, 2])
        if ((df.iloc[i, 0] - df.iloc[i-1, 0]) == 1) and ((df.iloc[i, 0] - df.iloc[i-2, 0]) == 2):
            
            A[prevState, state] += 1
        prevState = state
        
    for i in range(len(A)):
        totRow = np.sum(A[i,:])
        if totRow:
            A[i,:] = A[i,:] / totRow
    

    return A

I tabellen nedenfor finner man sannsynlighetene for de ulike overgangene mellom hver tilstand. Sannsynlighetene er beregnet utifra den tilgjengelige dataen. 

In [109]:
A2 = getProbs(df_states) #Get probabilities for model number 2

#Style and print the matrix
A2_pretty = styleMatrix(A2, ["SS (0)", "SO (1)", "SR (2)", "OS (3)", "OO (4)", "OR (5)", "RS (6)", "RO (7)", "RR (8)"])
A2_pretty.style.applymap(color_positive_blue)

,SS (0),SO (1),SR (2),OS (3),OO (4),OR (5),RS (6),RO (7),RR (8)
SS (0),0.2,0.133333,0.666667,0,0,0,0,0,0
SO (1),0,0,0,0.166667,0.266667,0.566667,0,0,0
SR (2),0,0,0,0,0,0,0.0503597,0.208633,0.741007
OS (3),0.225,0.15,0.625,0,0,0,0,0,0
OO (4),0,0,0,0.180723,0.349398,0.46988,0,0,0
OR (5),0,0,0,0,0,0,0.0723982,0.18552,0.742081
RS (6),0.313953,0.104651,0.581395,0,0,0,0,0,0
RO (7),0,0,0,0.223048,0.163569,0.613383,0,0,0
RR (8),0,0,0,0,0,0,0.0358142,0.111919,0.852266


Funksjonen nedenfor beregner hvor godt modellen klar spå fremtiden. Den returnerer en liste med andelen riktige spådommer for 1,2,...,n dager frem i tid.

In [112]:
def testProbs(A, df, daysIntoFuture):
    
    numbRows, numbCols = df.shape
    
    count = np.zeros(daysIntoFuture)
    correct = np.zeros(daysIntoFuture)
    prevStates = np.ones(daysIntoFuture) * (-1)
    for i in range(numbRows):
        currState = int(df.iloc[i, 2])
        
        #print("currState =", currState)
        
        if currState != -1:
            for j in range(len(prevStates)):
                predMatrix = multiplyArray(A, j)
                predState = np.argmax(predMatrix[int(prevStates[j])])

                #print("j =", j, "gir:", predState)

                if prevStates[j] == -1:
                    break

                if currState%3 == predState%3:
                    correct[j] += 1

                count[j] += 1
        
        try:
            diff = df.iloc[i+1,1] - df.iloc[i,1]
            leftShiftList(prevStates, diff.days)
        except:
            break #det er siste måling
        
        prevStates[0] = currState
        """

        print(prevStates)
        print(correct)
        print(count, "\n")
        """

    
    return correct / count

Henter ut testdata:

In [114]:
testDf2 = createDfStates(quickDfSimpleStates(2015, 2019))

I tabellen nedenfor finner man andelen riktige spådommer for hennholdsvis 1,2 og 3 dager frem i tid.

In [116]:
pd.DataFrame(testProbs(A2, testDf2, 3), columns = ["Andel riktig"], index = [1,2,3])

,Andel riktig
1,0.812606
2,0.814114
3,0.817391
